In [ ]:
"""multi_feature_svm — ELA / PRNU / CLIP pipeline (no-CLI)
=========================================================
將此格貼入 Jupyter Notebook。**所有可調整參數都在 CONFIG 區**—
修改後重新執行即可重新訓練。

流程：
1. 擷取並快取所選特徵 (ELA / PRNU / CLIP)
2. Concatenate → PCA → SVM (自動使用 GPU 後端若可用)
3. 輸出評估結果並存模型 `saved_models/multi_svm.pkl`
4. 附 `predict_one()` for 快速單張推論
"""

# ==================== Imports ====================
from __future__ import annotations
from pathlib import Path
from io import BytesIO
import os, time, importlib, warnings, random
import numpy as np
from PIL import Image, ImageChops, ImageFile
from skimage import io as skio, transform
from skimage.util import img_as_float32
from skimage.restoration import denoise_wavelet
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.svm import SVC as cpuSVC, LinearSVC
from sklearn.metrics import classification_report, confusion_matrix
import joblib

warnings.filterwarnings('ignore', category=FutureWarning)
ImageFile.LOAD_TRUNCATED_IMAGES = True

# ==================== CONFIG (edit values) ====================
DATA_ROOT = Path('/mnt/datasets/my_ela_train')
REAL_DIR  = DATA_ROOT / '/home/yaya/ai-detect-proj/data/raw/real/flickr30K'    # 真實照片資料夾
FAKE_DIR  = DATA_ROOT / '/home/yaya/ai-detect-proj/data/raw/fake/FLUX'        # AI 圖片資料夾

# 使用哪些特徵
USE_ELA  = True
USE_PRNU = True
USE_CLIP = True

CROP_SIZE = None
PCA_DIM   = 512      
# 降低 PCA 維度以節省 RAM
SAMPLE_PER_CLASS = 500    # ← 每類最多取 N 張；None = 全量
EXTRACT_PER_CLASS = 20000
  
IMG_SIZE = 512; IMG_QUALITY = 90; SCALE = 15; FEA_SIZE = 128
GPU_C, GPU_GAMMA = 10, 0.01; CPU_C, CPU_GAMMA = 10, 0.01; LIN_C = 1.0
TEST_RATIO = 0.2; SEED = 42; random.seed(SEED)

FEATURE_DIR = Path('features'); FEATURE_DIR.mkdir(exist_ok=True)
MODEL_PATH  = Path('saved_models/multi_svm.pkl'); MODEL_PATH.parent.mkdir(exist_ok=True)



In [ ]:
# ==================== Feature extractors ====================

def extract_prnu(p: Path, 
                 img_size: int = 512, 
                 amplify: float = 255.0, 
                 wavelet: str = "db8") -> np.ndarray | None:
    """
    回傳單通道 PRNU 殘差 (H, W)，失敗回 None。
    - 若影像較小，先雙線性放大到 ≥ img_size 再裁中心。
    - 先轉 float32∈[0,1]，避免 uint8 捨入。
    - 將殘差乘 amplify 以放大低振幅訊號。
    """
    try:
        # 讀檔並轉 float32
        img = skio.imread(str(p))
        if img.ndim == 2:                      # 灰階 → 疊成 3 通道便於後續 mean
            img = np.repeat(img[..., None], 3, -1)
        img = img_as_float32(img)

        # 尺寸不足就放大
        h, w = img.shape[:2]
        if min(h, w) < img_size:
            scale = img_size / min(h, w)
            new_h, new_w = round(h*scale), round(w*scale)
            img = skio.transform.resize(img, (new_h, new_w), anti_aliasing=True)
            h, w = img.shape[:2]

        # 中心裁 512×512
        sh, sw = h//2 - img_size//2, w//2 - img_size//2
        crop = img[sh:sh+img_size, sw:sw+img_size]

        # 取灰階
        gray = crop.mean(axis=2, dtype=np.float32)

        # 小波去雜訊（保留高頻）
        denoised = denoise_wavelet(gray, channel_axis=None, mode="soft",
                                   wavelet=wavelet, convert2ycbcr=False)

        residual = (gray - denoised) * amplify

        # 去掉 DC (加速後續 PCA / SVM)
        residual -= residual.mean()

        return residual.astype(np.float32)

    except Exception as e:
        print(f"[WARN] extract_prnu {p.name}: {e}")
        return None



def extract_ela(p: Path):
    """ELA 差分特徵；若尺寸不足 IMG_SIZE 則回傳 None"""
    try:
        img = Image.open(p).convert("RGB"); w, h = img.size
        if min(w, h) < IMG_SIZE:
            return None
        sw, sh = w // 2 - IMG_SIZE // 2, h // 2 - IMG_SIZE // 2
        crop = img.crop((sw, sh, sw + IMG_SIZE, sh + IMG_SIZE))

        buf = BytesIO()
        crop.save(buf, format="JPEG", quality=IMG_QUALITY)
        buf.seek(0)
        diff = ImageChops.difference(crop, Image.open(buf)).point(lambda x: x * SCALE)
        diff = diff.convert("L").resize((FEA_SIZE, FEA_SIZE))
        return np.asarray(diff, dtype=np.float32) / 255.0
    except Exception:
        return None


# -- Lazy-load CLIP ---------------------------------------------------------
_clip_model, _clip_pre = None, None


def load_clip():
    global _clip_model, _clip_pre
    if _clip_model is None:
        import torch, clip

        device = "cuda" if torch.cuda.is_available() else "cpu"
        _clip_model, _clip_pre = clip.load("ViT-L/14", device=device)
        _clip_model.eval()
    return _clip_model, _clip_pre


def extract_clip(p: Path):
    """CLIP ViT-B/32 向量；若圖片不足正方形則回傳 None"""
    try:
        import torch

        m, pre = load_clip()
        device = next(m.parameters()).device
        img = Image.open(p).convert("RGB")
        s = min(img.size); w, h = img.size
        img = img.crop(((w - s) // 2, (h - s) // 2, (w + s) // 2, (h + s) // 2))
        with torch.no_grad():
            vec = m.encode_image(pre(img).unsqueeze(0).to(device)).cpu().numpy()
        return vec.astype(np.float32)
    except Exception:
        return None


EXTRACTORS = {
    "ela": extract_ela,
    "prnu": extract_prnu,
    "clip": extract_clip,
}
SELECTED = [
    k
    for k, flag in [
        ("ela", USE_ELA),
        ("prnu", USE_PRNU),
        ("clip", USE_CLIP),
    ]
    if flag  # Add a boolean expression here
    
]

# ==================== Prepare & cache features ====================
print("🔄 Extracting features:", SELECTED)
rng = random.Random(SEED)      # 確保抽樣可重現

# ==================== Prepare & cache features (recursive, robust) ====================
from tqdm import tqdm
import mimetypes, shutil

def all_images(root: Path) -> list[Path]:
    """遞迴抓 root 下面所有圖片檔 (.jpg/.png/.jpeg)；忽略壓縮與其它副檔。"""
    exts = {'.jpg', '.jpeg', '.png'}
    return [p for p in root.rglob('*') if p.suffix.lower() in exts]

def _slug(s: str) -> str:
    # 安全化：只保留英數/ - _ .，其他字元轉成 _
    return "".join(c if (c.isalnum() or c in "-_.") else "_" for c in s)

def make_id(p: Path, root: Path) -> str:
    """
    原本：a/b/c.jpg -> a_b_c
    現在：<dataset>__a_b_c
    其中 <dataset> = root.name（例如 unsplash、FLUX）
    """
    rel = p.relative_to(root)                 # a/b/c.jpg
    base = "_".join(rel.with_suffix("").parts)
    dataset = _slug(root.name)                # e.g., "unsplash" or "FLUX"
    return f"{dataset}__{base}"

print("🔄 Extracting & syncing features :", SELECTED)
rng = random.Random(SEED)

# 1) 目的資料夾
for feat in SELECTED:
    for cls in ["real", "fake"]:
        (FEATURE_DIR / f"{feat}_{cls}").mkdir(parents=True, exist_ok=True)

# 2) 來源清單
src_roots = {
    "real": Path("/home/yaya/ai-detect-proj/data/raw/real"),
    "fake": Path("/home/yaya/ai-detect-proj/data/raw/fake"),
}

for cls, root in src_roots.items():
    files = all_images(root)
    if EXTRACT_PER_CLASS and len(files) > EXTRACT_PER_CLASS:
        files = rng.sample(files, EXTRACT_PER_CLASS)

    dst_dirs = {feat: FEATURE_DIR / f"{feat}_{cls}" for feat in SELECTED}

    for img in tqdm(files, desc=f"{cls}"):
        fid = make_id(img, root)                       # unique id

        # 若三特徵檔都已存在 → 跳過
        if all((dst_dirs[f]/f"{fid}.npy").exists() for f in SELECTED):
            continue

        ok, vecs = True, {}
        for feat in SELECTED:
            vec = EXTRACTORS[feat](img)
            if vec is None:
                ok = False; break
            vecs[feat] = vec

        if ok:     # 全部成功 → 寫檔
            for feat, arr in vecs.items():
                np.save(dst_dirs[feat] / f"{fid}.npy", arr)
        else:      # 有失敗 → 清掉殘檔
            for feat in SELECTED:
                tmp = dst_dirs[feat] / f"{fid}.npy"
                if tmp.exists(): tmp.unlink()

print("✅ 全 real / fake 目錄已遞迴處理；Feature 資料夾保持同步")



🔄 Extracting features: ['ela', 'prnu', 'clip']
🔄 Extracting & syncing features : ['ela', 'prnu', 'clip']


real:   3%|▎         | 694/20006 [00:08<04:06, 78.26it/s]/home/yaya/miniforge3/envs/imgfeat/lib/python3.11/site-packages/PIL/Image.py:3452: DecompressionBombWarning: Image size (121500000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/home/yaya/miniforge3/envs/imgfeat/lib/python3.11/site-packages/PIL/Image.py:3452: DecompressionBombWarning: Image size (121500000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/home/yaya/miniforge3/envs/imgfeat/lib/python3.11/site-packages/PIL/Image.py:3452: DecompressionBombWarning: Image size (121500000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
real:   5%|▌         | 1050/20006 [03:42<3:08:18,  1.68it/s]/home/yaya/miniforge3/envs/imgfeat/lib/python3.11/site-packages/PIL/Image.py:3452: DecompressionBombWarning: Image size (99996755 pixels) exceeds limit of 89478485 pixels, could be decompressio

✅ 全 real / fake 目錄已遞迴處理；Feature 資料夾保持同步


In [1]:
# ======================================================
# Extract ELA / PRNU / CLIP features → save as .npy
# ======================================================

from pathlib import Path
from io import BytesIO
import os, random, warnings
import numpy as np
from PIL import Image, ImageChops, ImageFile
from skimage import io as skio
from skimage.util import img_as_float32
from skimage.restoration import denoise_wavelet
from skimage.transform import resize
from tqdm import tqdm

warnings.filterwarnings("ignore", category=UserWarning)
ImageFile.LOAD_TRUNCATED_IMAGES = True

# ---------------- Config（改這裡） ----------------
REAL_DIR = Path("/home/yaya/ai-detect-proj/data/raw/real/flickr30K")
FAKE_DIR = Path("/home/yaya/ai-detect-proj/data/raw/fake/dalle3")

OUT_ROOT = Path("/home/yaya/ai-detect-proj/Script/features_256")  # 會自動建立
# 會生成：
#   OUT_ROOT/ela_real_npy/*.npy
#   OUT_ROOT/ela_fake_npy/*.npy
#   OUT_ROOT/prnu_real_npy/*.npy
#   OUT_ROOT/prnu_fake_npy/*.npy
#   OUT_ROOT/clip_real_npy/*.npy
#   OUT_ROOT/clip_fake_npy/*.npy

RUN_CLASSES = ["fake"]   # 你也可以填 ["real"] 或 ["fake"] 只跑其中一類
SELECT_FEATURES = ["ela", "prnu", "clip"]  # 想只跑其中幾個就刪掉其餘

# ELA 參數
IMG_SIZE     = 265     # 先把最短邊放到 >= 這個長度後做中心裁切
ELA_QUALITY  = 90
ELA_SCALE    = 15      # 只是把差值放大以增強對比，輸出仍會正規化到 0..1
ELA_FEASZ    = 128     # ELA 輸出尺寸

# PRNU 參數
PRNU_CROP_FROM = 265   # 中心裁起始邊長（不足會放大）
PRNU_OUT_SIZE  = 256   # PRNU 輸出尺寸
PRNU_WAVELET   = "db8" # 小波基
PRNU_MODE      = "soft"

# CLIP 模型（需要 pip 安裝 openai-clip；會 lazy-load）
CLIP_MODEL_NAME = "ViT-L/14"  # 你也可用 "ViT-B/32"

SEED = 42
random.seed(SEED); np.random.seed(SEED)

# ---------------- Utils ----------------
IMG_EXTS = {".jpg", ".jpeg", ".png", ".webp", ".bmp", ".tif", ".tiff"}

def all_images(root: Path):
    return [p for p in root.rglob("*") if p.suffix.lower() in IMG_EXTS]

def ensure_dirs():
    OUT_ROOT.mkdir(parents=True, exist_ok=True)
    for feat in SELECT_FEATURES:
        for cls in RUN_CLASSES:
            (OUT_ROOT / f"{feat}_{cls}_npy").mkdir(parents=True, exist_ok=True)

def center_resize_crop_PIL(img: Image.Image, to_size: int) -> Image.Image:
    w, h = img.size
    if min(w, h) < to_size:
        s = to_size / min(w, h)
        img = img.resize((int(round(w*s)), int(round(h*s))), Image.BICUBIC)
        w, h = img.size
    x0, y0 = (w - to_size)//2, (h - to_size)//2
    return img.crop((x0, y0, x0 + to_size, y0 + to_size))

def center_resize_crop_np(img: np.ndarray, crop_from=512, out_size=256) -> np.ndarray:
    h, w = img.shape[:2]
    if min(h, w) < crop_from:
        s = crop_from / min(h, w)
        img = resize(img, (int(round(h*s)), int(round(w*s))), preserve_range=True, anti_aliasing=True).astype(img.dtype)
        h, w = img.shape[:2]
    y0, x0 = (h - crop_from)//2, (w - crop_from)//2
    img = img[y0:y0+crop_from, x0:x0+crop_from]
    if crop_from != out_size:
        img = resize(img, (out_size, out_size), preserve_range=True, anti_aliasing=True).astype(img.dtype)
    return img

# ---------------- Extractors ----------------
def extract_ela_arr(p: Path) -> np.ndarray | None:
    """回傳 float32 (H,W) ∈ [0,1]，大小為 ELA_FEASZ×ELA_FEASZ"""
    try:
        img = Image.open(p).convert("RGB")
        img = center_resize_crop_PIL(img, IMG_SIZE)
        # JPEG 重壓 & 差分
        buf = BytesIO()
        img.save(buf, format="JPEG", quality=int(ELA_QUALITY), subsampling=0, optimize=False)
        buf.seek(0)
        diff = ImageChops.difference(img, Image.open(buf)).point(lambda x: x * ELA_SCALE)
        diff = diff.convert("L").resize((ELA_FEASZ, ELA_FEASZ))
        arr = np.asarray(diff, dtype=np.float32) / 255.0
        return arr
    except Exception as e:
        print("[ELA] skip", p.name, "|", e)
        return None

def extract_prnu_arr(p: Path) -> np.ndarray | None:
    """回傳 float32 (PRNU_OUT_SIZE, PRNU_OUT_SIZE)，零均值"""
    try:
        im = skio.imread(str(p))
        if im.ndim == 2:
            im = np.repeat(im[..., None], 3, axis=-1)
        im = img_as_float32(im)  # 0..1
        crop = center_resize_crop_np(im, PRNU_CROP_FROM, PRNU_OUT_SIZE)
        gray = crop.mean(axis=2, dtype=np.float32)
        denoised = denoise_wavelet(gray, channel_axis=None, mode=PRNU_MODE,
                                   wavelet=PRNU_WAVELET, convert2ycbcr=False)
        residual = gray - denoised
        residual -= residual.mean()
        return residual.astype(np.float32)
    except Exception as e:
        print("[PRNU] skip", p.name, "|", e)
        return None

_clip_model, _clip_pre = None, None
def load_clip():
    global _clip_model, _clip_pre
    if _clip_model is None:
        import torch, clip
        dev = "cuda" if torch.cuda.is_available() else "cpu"
        _clip_model, _clip_pre = clip.load(CLIP_MODEL_NAME, device=dev)
        _clip_model.eval()
    return _clip_model, _clip_pre

def extract_clip_vec(p: Path) -> np.ndarray | None:
    """回傳 float32 向量 (D,)；長度依 CLIP 模型而定"""
    try:
        import torch
        m, pre = load_clip()
        dev = next(m.parameters()).device
        img = Image.open(p).convert("RGB")
        # 讓圖片近似正方 → 中心裁最短邊
        s = min(img.size); w, h = img.size
        img = img.crop(((w - s)//2, (h - s)//2, (w + s)//2, (h + s)//2))
        with torch.no_grad():
            vec = m.encode_image(pre(img).unsqueeze(0).to(dev)).float()
            vec = vec / vec.norm(dim=-1, keepdim=True)  # L2 normalize（常見做法）
            arr = vec.cpu().numpy().reshape(-1).astype(np.float32)
        return arr
    except Exception as e:
        print("[CLIP] skip", p.name, "|", e)
        return None

# ---------------- Runner ----------------
def _slug(s: str) -> str:
    # 安全化：只保留英數/ - _ .，其他字元轉成 _
    return "".join(c if (c.isalnum() or c in "-_.") else "_" for c in s)

def make_id(p: Path, root: Path) -> str:
    """
    原本：a/b/c.jpg -> a_b_c
    現在：<dataset>__a_b_c
    其中 <dataset> = root.name（例如 unsplash、FLUX）
    """
    rel = p.relative_to(root)                 # a/b/c.jpg
    base = "_".join(rel.with_suffix("").parts)
    dataset = _slug(root.name)                # e.g., "unsplash" or "FLUX"
    return f"{dataset}__{base}"

def run_extract_for_class(cls: str, root: Path):
    files = all_images(root)
    print(f"→ {cls} ({len(files)} images)")

    dirs = {}
    if "ela" in SELECT_FEATURES:  dirs["ela"]  = OUT_ROOT / f"ela_{cls}_npy"
    if "prnu" in SELECT_FEATURES: dirs["prnu"] = OUT_ROOT / f"prnu_{cls}_npy"
    if "clip" in SELECT_FEATURES: dirs["clip"] = OUT_ROOT / f"clip_{cls}_npy"

    for d in dirs.values(): d.mkdir(parents=True, exist_ok=True)

    for img in tqdm(files, desc=f"extract {cls}"):
        fid = make_id(img, root)

        if "ela" in dirs:
            out = dirs["ela"] / f"{fid}.npy"
            if not out.exists():
                arr = extract_ela_arr(img)
                if arr is not None: np.save(out, arr)

        if "prnu" in dirs:
            out = dirs["prnu"] / f"{fid}.npy"
            if not out.exists():
                arr = extract_prnu_arr(img)
                if arr is not None: np.save(out, arr)

        if "clip" in dirs:
            out = dirs["clip"] / f"{fid}.npy"
            if not out.exists():
                arr = extract_clip_vec(img)
                if arr is not None: np.save(out, arr)

# ---------------- Go! ----------------
ensure_dirs()
if "real" in RUN_CLASSES: run_extract_for_class("real", REAL_DIR)
if "fake" in RUN_CLASSES: run_extract_for_class("fake", FAKE_DIR)
print("✅ Done. Features saved under:", OUT_ROOT)


→ fake (19000 images)


extract fake:  12%|█▏        | 2364/19000 [03:57<27:47,  9.97it/s]  


KeyboardInterrupt: 

In [8]:
# ==================== 0. 清理資料夾 ====================
import os, numpy as np
from pathlib import Path
from tqdm import tqdm

def is_bad(arr, expect_shape):
    """判斷單一 ndarray 是否不合法"""
    if arr.shape != expect_shape:
        return True
    if np.isnan(arr).any() or np.isinf(arr).any():
        return True
    if arr.std() < 1e-6:   # 幾乎常數
        return True
    return False

def clean_feature_dirs(root: Path):
    """
    針對 clip_* / prnu_* / ela_* 三資料夾：
      1. 取交集 id
      2. 若任一檔案缺失 / 資料異常 → 同步刪除三份 .npy
    """
    type_shape = {
        "clip": (1, 768),
        "prnu": (512, 512),
        "ela":  (128, 128),
    }

    for cls in ["real", "fake"]:
        dirs = {t: root/f"{t}_{cls}" for t in type_shape}
        ids  = set.intersection(*[
            {p.stem for p in dirs[t].glob("*.npy")} for t in type_shape
        ])

        removed = 0
        for fid in tqdm(ids, desc=f"clean-{cls}"):
            bad = False
            for t, shp in type_shape.items():
                fpath = dirs[t]/f"{fid}.npy"
                try:
                    arr = np.load(fpath)
                    if is_bad(arr, shp):
                        bad = True; break
                except Exception:
                    bad = True; break

            if bad:
                removed += 1
                for t in type_shape:
                    f = dirs[t]/f"{fid}.npy"
                    if f.exists(): os.remove(f)
        print(f"[{cls}] removed {removed} bad samples")

# ---------- 呼叫一次即可 ----------
DATA_ROOT = Path("../Script/features_npy")
clean_feature_dirs(DATA_ROOT)


clean-real: 0it [00:00, ?it/s]


[real] removed 0 bad samples


clean-fake: 0it [00:00, ?it/s]

[fake] removed 0 bad samples


In [11]:
# ===== 將所有 real 資料夾的 .npy 加上 imagenet__ 前綴 =====
import os, re, json, time
from pathlib import Path

SCRIPT_ROOT = Path("/home/yaya/ai-detect-proj/Script")
FEATURE_ROOT = SCRIPT_ROOT / "features_npy"
REAL_DIRS = ["clip_real_npy", "ela_real_npy", "prnu_real_npy"]

DELIM = "__"
DRY_RUN = False         # 先設 True 看輸出沒問題再改 False 正式改名
ONLY_IF_NO_PREFIX = True  # True: 只有當檔名沒有任何前綴時才加 imagenet__

OUTPUT_DIR = SCRIPT_ROOT / "saved_models"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

pat_imagenet = re.compile(r'(?i)^imagenet(?:(?:__|---)|[_\-\s])')
# 偵測「已有其它前綴」：遇到分隔符前若有英數字母就算有前綴（排除純數字序號）
SEPS = ("__", "---", "--", "_", "-", " ")
def has_any_prefix(stem: str) -> bool:
    cut = None
    for s in SEPS:
        i = stem.find(s)
        if i != -1:
            cut = i if cut is None else min(cut, i)
    if cut is None:
        return False
    left = stem[:cut].strip()
    return bool(left) and not left.isdigit()

def unique_target(base: Path) -> Path:
    if not base.exists():
        return base
    stem, suf = base.stem, base.suffix
    n = 1
    while True:
        cand = base.with_name(f"{stem}__dup{n}{suf}")
        if not cand.exists():
            return cand
        n += 1

summary = {}
ts = time.strftime("%Y%m%d_%H%M%S")
log_path = OUTPUT_DIR / f"rename_imagenet_allreal_{ts}.json"
mapping_all = []

for dname in REAL_DIRS:
    dpath = FEATURE_ROOT / dname
    assert dpath.is_dir(), f"不存在：{dpath}"
    files = sorted(p for p in dpath.glob("*.npy") if p.is_file())
    renamed, skipped = [], []

    for src in files:
        stem = src.stem

        # 已是 imagenet 前綴 → 跳過
        if pat_imagenet.match(stem):
            skipped.append({"name": src.name, "reason": "already_imagenet"})
            continue

        # 若只在「沒有任何前綴」時才加，且偵測到已有其它前綴 → 跳過
        if ONLY_IF_NO_PREFIX and has_any_prefix(stem):
            skipped.append({"name": src.name, "reason": "has_other_prefix"})
            continue

        dst = unique_target(src.with_name(f"imagenet{DELIM}{src.name}"))
        if DRY_RUN:
            print(f"[DRY] {src.relative_to(FEATURE_ROOT)} -> {dst.relative_to(FEATURE_ROOT)}")
        else:
            src.rename(dst)
        renamed.append({"from": str(src.relative_to(FEATURE_ROOT)),
                        "to":   str(dst.relative_to(FEATURE_ROOT))})

    mapping_all.append({"dir": dname, "renamed": renamed, "skipped": skipped})
    summary[dname] = {"renamed": len(renamed), "skipped": len(skipped)}

# 寫入對照表（一次性涵蓋三個資料夾）
with open(log_path, "w", encoding="utf-8") as f:
    json.dump(mapping_all, f, ensure_ascii=False, indent=2)

print("完成各資料夾統計：")
for k, v in summary.items():
    print(f"- {k}: renamed={v['renamed']} skipped={v['skipped']}")
print(f"對照表：{log_path}")


完成各資料夾統計：
- clip_real_npy: renamed=60000 skipped=0
- ela_real_npy: renamed=60000 skipped=0
- prnu_real_npy: renamed=60000 skipped=0
對照表：/home/yaya/ai-detect-proj/Script/saved_models/rename_imagenet_allreal_20250813_213602.json
